In [73]:
import platform
import numpy as np
import pandas as pd
import random

import torch 
from torch import optim 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn

from tqdm.notebook import tqdm
from transformers import AutoTokenizer

# enable tqdm in pandas
tqdm.pandas()

# select device
if torch.cuda.is_available():
    device = torch.device('cuda')
elif 'arm64' in platform.platform():
    device = torch.device('mps') # 'mps'
else:
    device = torch.device('cpu')
print(f'device: {device.type}') 

# random seed
seed = 1234

# pytorch ignores this label in the loss
ignore_index = -100

# set random seed
if seed is not None:
    print(f'random seed: {seed}')
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

# which transformer to use
transformer_name = "bert-base-cased" # 'xlm-roberta-base' # 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(transformer_name)

device: mps
random seed: 1234


In [74]:

# map labels to the first token in each word
def align_labels(word_ids, labels, label_to_index):
    label_ids = []
    previous_word_id = None
    for word_id in word_ids:
        if word_id is None or word_id == previous_word_id:
            # ignore if not a word or word id has already been seen
            label_ids.append(ignore_index)
        else:
            # get label id for corresponding word
            label_id = label_to_index[labels[word_id]]
            label_ids.append(label_id)
        # remember this word id
        previous_word_id = word_id
    
    return label_ids
            
# build a set of labels in the dataset            
def read_label_set(fn):
    labels = set()
    with open(fn) as f:
        for index, line in enumerate(f):
            line = line.strip()
            tokens = line.split()
            if tokens != []:
                label = tokens[-1]
                labels.add(label)
    return labels

# converts a two-column file in the basic MTL format ("word \t label") into a dataframe
def read_dataframe(fn, label_to_index, task_id):
    # now build the actual dataframe for this dataset
    data = {'words': [], 'str_labels': [], 'input_ids': [], 'word_ids': [], 'labels': [], 'task_ids': []}
    with open(fn) as f:
        sent_words = []
        sent_labels = [] 
        for index, line in tqdm(enumerate(f)):
            line = line.strip()
            tokens = line.split()
            if tokens == []:
                data['words'].append(sent_words)
                data['str_labels'].append(sent_labels)
                
                # tokenize each sentence
                token_input = tokenizer(sent_words, is_split_into_words = True)  
                token_ids = token_input['input_ids']
                word_ids = token_input.word_ids(batch_index = 0)
                
                # map labels to the first token in each word
                token_labels = align_labels(word_ids, sent_labels, label_to_index)
                
                data['input_ids'].append(token_ids)
                data['word_ids'].append(word_ids)
                data['labels'].append(token_labels)
                data['task_ids'].append(task_id)
                sent_words = []
                sent_labels = [] 
            else:
                sent_words.append(tokens[0])
                sent_labels.append(tokens[1])
    return pd.DataFrame(data)


In [75]:
class Task():
    def __init__(self, task_id, train_file_name, dev_file_name, test_file_name):
        self.task_id = task_id
        # we need an index of labels first
        self.labels = read_label_set(train_file_name)
        self.index_to_label = {i:t for i,t in enumerate(self.labels)}
        self.label_to_index = {t:i for i,t in enumerate(self.labels)}
        self.num_labels = len(self.index_to_label)
        # create data frames for the datasets
        self.train_df = read_dataframe(train_file_name, self.label_to_index, self.task_id)
        self.dev_df = read_dataframe(dev_file_name, self.label_to_index, self.task_id)
        self.test_df = read_dataframe(test_file_name, self.label_to_index, self.task_id)
                

In [76]:
ner_task = Task(0, "data/conll-ner/train_small.txt", "data/conll-ner/dev.txt", "data/conll-ner/test.txt")
pos_task = Task(1, "data/pos/train_small.txt", "data/pos/dev.txt", "data/pos/test.txt")

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [77]:
ner_task.train_df

,words,str_labels,input_ids,word_ids,labels,task_ids
0,"[EU, rejects, German, call, to, boycott, Briti...","[B-ORG, O, B-MISC, O, O, O, B-MISC, O, O]","[101, 7270, 22961, 1528, 1840, 1106, 21423, 14...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]","[-100, 4, 7, 8, 7, 7, 7, 8, 7, -100, 7, -100]",0
1,"[Peter, Blackburn]","[B-PER, I-PER]","[101, 1943, 14428, 102]","[None, 0, 1, None]","[-100, 0, 2, -100]",0
2,"[BRUSSELS, 1996-08-22]","[B-LOC, O]","[101, 26660, 13329, 12649, 15928, 1820, 118, 4...","[None, 0, 0, 0, 0, 1, 1, 1, 1, 1, None]","[-100, 3, -100, -100, -100, 7, -100, -100, -10...",0
3,"[The, European, Commission, said, on, Thursday...","[O, B-ORG, I-ORG, O, O, O, O, O, O, B-MISC, O,...","[101, 1109, 1735, 2827, 1163, 1113, 9170, 1122...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...","[-100, 7, 4, 5, 7, 7, 7, 7, 7, 7, 8, 7, 7, 7, ...",0
4,"[Germany, 's, representative, to, the, Europea...","[B-LOC, O, O, O, O, B-ORG, I-ORG, O, O, O, B-P...","[101, 1860, 112, 188, 4702, 1106, 1103, 1735, ...","[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10,...","[-100, 3, 7, -100, 7, 7, 7, 4, 5, 7, -100, 7, ...",0
5,"["", We, do, n't, support, any, such, recommend...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[101, 107, 1284, 1202, 183, 112, 189, 1619, 12...","[None, 0, 1, 2, 3, 3, 3, 4, 5, 6, 7, 8, 9, 10,...","[-100, 7, 7, 7, 7, -100, -100, 7, 7, 7, 7, 7, ...",0
6,"[He, said, further, scientific, study, was, re...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[101, 1124, 1163, 1748, 3812, 2025, 1108, 2320...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...","[-100, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...",0
7,"[He, said, a, proposal, last, month, by, EU, F...","[O, O, O, O, O, O, O, B-ORG, O, O, B-PER, I-PE...","[101, 1124, 1163, 170, 5835, 1314, 2370, 1118,...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...","[-100, 7, 7, 7, 7, 7, 7, 7, 4, 7, 7, 0, 2, -10...",0
8,"[Fischler, proposed, EU-wide, measures, after,...","[B-PER, O, B-MISC, O, O, O, O, B-LOC, O, B-LOC...","[101, 17355, 9022, 2879, 3000, 7270, 118, 2043...","[None, 0, 0, 0, 1, 2, 2, 2, 3, 4, 5, 6, 7, 8, ...","[-100, 0, -100, -100, 7, 8, -100, -100, 7, 7, ...",0
9,"[But, Fischler, agreed, to, review, his, propo...","[O, B-PER, O, O, O, O, O, O, O, B-ORG, O, O, O...","[101, 1252, 17355, 9022, 2879, 2675, 1106, 318...","[None, 0, 1, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,...","[-100, 7, 0, -100, -100, 7, 7, 7, 7, 7, 7, 7, ...",0


In [78]:
pos_task.train_df

,words,str_labels,input_ids,word_ids,labels,task_ids
0,"[Pierre, Vinken, ,, 61, years, old, ,, will, j...","[NNP, NNP, ,, CD, NNS, JJ, ,, MD, VB, DT, NN, ...","[101, 4855, 25354, 6378, 117, 5391, 1201, 1385...","[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11...","[-100, 29, 29, -100, 7, 31, 0, 27, 7, 34, 10, ...",1
1,"[Mr., Vinken, is, chairman, of, Elsevier, N.V....","[NNP, NNP, VBZ, NN, IN, NNP, NNP, ,, DT, NNP, ...","[101, 1828, 119, 25354, 6378, 1110, 3931, 1104...","[None, 0, 0, 1, 1, 2, 3, 4, 5, 5, 5, 6, 6, 6, ...","[-100, 29, -100, 29, -100, 18, 28, 24, 29, -10...",1
2,"[Rudolph, Agnew, ,, 55, years, old, and, forme...","[NNP, NNP, ,, CD, NNS, JJ, CC, JJ, NN, IN, NNP...","[101, 19922, 138, 8376, 2246, 117, 3731, 1201,...","[None, 0, 1, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,...","[-100, 29, 29, -100, -100, 7, 31, 0, 27, 44, 2...",1
3,"[A, form, of, asbestos, once, used, to, make, ...","[DT, NN, IN, NN, RB, VBN, TO, VB, NNP, NN, NNS...","[101, 138, 1532, 1104, 1112, 12866, 11990, 151...","[None, 0, 1, 2, 3, 3, 3, 4, 5, 6, 7, 8, 9, 10,...","[-100, 13, 28, 24, 28, -100, -100, 33, 25, 20,...",1
4,"[The, asbestos, fiber, ,, crocidolite, ,, is, ...","[DT, NN, NN, ,, NN, ,, VBZ, RB, JJ, IN, PRP, V...","[101, 1109, 1112, 12866, 11990, 12753, 117, 17...","[None, 0, 1, 1, 1, 2, 3, 4, 4, 4, 4, 4, 5, 6, ...","[-100, 13, 28, -100, -100, 28, 7, 28, -100, -1...",1
...,...,...,...,...,...,...
196,"[On, the, other, hand, ,, had, it, existed, th...","[IN, DT, JJ, NN, ,, VBD, PRP, VBN, RB, ,, NNP,...","[101, 1212, 1103, 1168, 1289, 117, 1125, 1122,...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 1...","[-100, 24, 13, 27, 28, 7, 43, 1, 25, 33, 7, 29...",1
197,"[Mr., Cray, ,, who, could, n't, be, reached, f...","[NNP, NNP, ,, WP, MD, RB, VB, VBN, IN, NN, ,, ...","[101, 1828, 119, 140, 6447, 117, 1150, 1180, 1...","[None, 0, 0, 1, 1, 2, 3, 4, 5, 5, 5, 6, 7, 8, ...","[-100, 29, -100, 29, -100, 7, 15, 34, 33, -100...",1
198,"[Regarded, as, the, father, of, the, supercomp...","[VBN, IN, DT, NN, IN, DT, NN, ,, NNP, NNP, VBD...","[101, 23287, 26541, 1112, 1103, 1401, 1104, 11...","[None, 0, 0, 1, 2, 3, 4, 5, 6, 6, 6, 6, 7, 8, ...","[-100, 25, -100, 24, 13, 28, 24, 13, 28, -100,...",1
199,"[At, Cray, Computer, ,, he, will, be, paid, $,...","[IN, NNP, NNP, ,, PRP, MD, VB, VBN, $, CD, .]","[101, 1335, 140, 6447, 6701, 117, 1119, 1209, ...","[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 9, ...","[-100, 24, 29, -100, 29, 7, 1, 34, 10, 25, 3, ...",1


In [95]:
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.bert.modeling_bert import BertModel, BertPreTrainedModel
from transformers import PreTrainedModel
from transformers import AutoConfig, AutoModel

# This class is adapted from: https://towardsdatascience.com/how-to-create-and-train-a-multi-task-transformer-model-18c54a146240
class TokenClassificationModel(BertPreTrainedModel):    
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config, add_pooling_layer=False)
        self.output_heads = nn.ModuleDict() # these are initialized in add_heads
        self.init_weights()
        
    def add_heads(self, tasks):
        for task in tasks:
            head = TokenClassificationHead(self.bert.config.hidden_size, task.num_labels, config.hidden_dropout_prob)
            # ModuleDict requires keys to be strings
            self.output_heads[str(task.task_id)] = head
        return self
    
    def summarize_heads(self):
        print(f'Found {len(self.output_heads)} heads')
        for task_id in self.output_heads:
            self.output_heads[task_id].summarize(task_id)
        
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, task_ids=None, **kwargs):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            **kwargs,
        )
        sequence_output = outputs[0]
        
        #print(f'batch size = {len(input_ids)}')
        #print(f'task_ids in this batch: {task_ids}')
        
        # generate specific predictions and losses for each task head
        unique_task_ids_list = torch.unique(task_ids).tolist()
        logits = None
        loss_list = []
        for unique_task_id in unique_task_ids_list:
            task_id_filter = task_ids == unique_task_id
            filtered_sequence_output = sequence_output[task_id_filter]
            filtered_labels = None if labels is None else labels[task_id_filter]
            filtered_attention_mask = None if attention_mask is None else attention_mask[task_id_filter]
            #print(f'size of batch for task {unique_task_id} is: {len(filtered_sequence_output)}')
            logits, task_loss = self.output_heads[str(unique_task_id)].forward(
                filtered_sequence_output, None,
                filtered_labels,
                filtered_attention_mask,
            )
            if filtered_labels is not None:
                loss_list.append(task_loss)
                
        loss = None if len(loss_list) == 0 else torch.stack(loss_list)
                    
        # logits are only used for eval, in which case we handle a single task at a time
        # TODO: allow all tasks in the forward pass at inference                     
        return TokenClassifierOutput(
            loss=loss.mean(),
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

class TokenClassificationHead(nn.Module):
    def __init__(self, hidden_size, num_labels, dropout_p=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout_p)
        self.classifier = nn.Linear(hidden_size, num_labels)
        self.num_labels = num_labels

        self._init_weights()

    def _init_weights(self):
        self.classifier.weight.data.normal_(mean=0.0, std=0.02)
        if self.classifier.bias is not None:
            self.classifier.bias.data.zero_()
            
    def summarize(self, task_id):
        print(f"Task {task_id} with {self.num_labels} labels.")
        print(f'Dropout is {self.dropout}')
        print(f'Classifier layer is {self.classifier}')

    def forward(self, sequence_output, pooled_output, labels=None, attention_mask=None, **kwargs):
        sequence_output_dropout = self.dropout(sequence_output)
        logits = self.classifier(sequence_output_dropout)
        
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()            
            inputs = logits.view(-1, self.num_labels)
            targets = labels.view(-1)
            loss = loss_fn(inputs, targets)

        return logits, loss

In [96]:
tasks = [ner_task, pos_task]
config = AutoConfig.from_pretrained(transformer_name)
model= TokenClassificationModel.from_pretrained(transformer_name, config=config).add_heads(tasks)
model.summarize_heads()

Found 2 heads
Task 0 with 9 labels.
Dropout is Dropout(p=0.1, inplace=False)
Classifier layer is Linear(in_features=768, out_features=9, bias=True)
Task 1 with 45 labels.
Dropout is Dropout(p=0.1, inplace=False)
Classifier layer is Linear(in_features=768, out_features=45, bias=True)


In [90]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    # gold labels
    label_ids = eval_pred.label_ids
    # predictions
    pred_ids = np.argmax(eval_pred.predictions, axis=-1)
    # collect gold and predicted labels, ignoring ignore_index label
    y_true, y_pred = [], []
    batch_size, seq_len = pred_ids.shape
    for i in range(batch_size):
        for j in range(seq_len):
            if label_ids[i, j] != ignore_index:
                y_true.append(label_ids[i][j]) #index_to_label[label_ids[i][j]])
                y_pred.append(pred_ids[i][j]) #index_to_label[pred_ids[i][j]])
    # return computed metrics
    return {'accuracy': accuracy_score(y_true, y_pred)}

In [82]:
from datasets import Dataset, DatasetDict

ds = DatasetDict()
ds['train'] = Dataset.from_pandas(pd.concat([ner_task.train_df, pos_task.train_df]))
ds['validation'] = Dataset.from_pandas(pd.concat([ner_task.dev_df, pos_task.dev_df]))
ds['test'] = Dataset.from_pandas(pd.concat([ner_task.test_df, pos_task.test_df]))

# these are no longer needed; discard them to save memory
ner_task.train_df = None
pos_task.train_df = None

ds

DatasetDict({
    train: Dataset({
        features: ['words', 'str_labels', 'input_ids', 'word_ids', 'labels', 'task_ids', '__index_level_0__'],
        num_rows: 244
    })
    validation: Dataset({
        features: ['words', 'str_labels', 'input_ids', 'word_ids', 'labels', 'task_ids', '__index_level_0__'],
        num_rows: 8504
    })
    test: Dataset({
        features: ['words', 'str_labels', 'input_ids', 'word_ids', 'labels', 'task_ids', '__index_level_0__'],
        num_rows: 6101
    })
})

In [97]:
def evaluate(trainer, task, name):
    print(f"Evaluating on the validation dataset for task {name}:")
    ds = Dataset.from_pandas(task.dev_df)
    scores = trainer.evaluate(ds)
    return scores

In [98]:
from transformers import TrainingArguments
from transformers import Trainer
from transformers import DataCollatorForTokenClassification
import time
from datetime import timedelta

epochs = 2
batch_size = 128
weight_decay = 0.01
use_mps_device = True if str(device) == 'mps' else False
model_name = f'{transformer_name}-mtl'

data_collator = DataCollatorForTokenClassification(tokenizer)
last_checkpoint = None

for epoch in range(1, epochs + 1):
    print(f'STARTING EPOCH {epoch}')
    if last_checkpoint != None:
        print(f'Resuming from checkpoint {last_checkpoint}')
            
    training_args = TrainingArguments(
        output_dir=model_name,
        log_level='error',
        num_train_epochs=1, # one epoch at a time
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        # evaluation_strategy='epoch',
        do_eval=False, # we will evaluate each task explicitly
        weight_decay=weight_decay,
        resume_from_checkpoint = last_checkpoint,
        use_mps_device = use_mps_device
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        # compute_metrics=compute_metrics,
        train_dataset=ds['train'],
        # eval_dataset=ds['validation'],
        tokenizer=tokenizer
    )
    
    model.summarize_heads()

    start_time = time.monotonic()
    trainer.train()
    end_time = time.monotonic()
    print(f"Elapsed time for epoch {epoch}: {timedelta(seconds=end_time - start_time)}")

    ner_scores = evaluate(trainer, ner_task, "NER")
    pos_scores = evaluate(trainer, pos_task, "POS")
    macro_loss = (ner_scores['eval_loss'] + pos_scores['eval_loss'])/2
    print(f'DEV MACRO LOSS FOR EPOCH {epoch}: {macro_loss}\n\n')

    last_checkpoint = training_args.output_dir + '/mtl_model_epoch' + str(epoch)
    trainer.save_model(last_checkpoint)

STARTING EPOCH 1
Found 2 heads
Task 0 with 9 labels.
Dropout is Dropout(p=0.1, inplace=False)
Classifier layer is Linear(in_features=768, out_features=9, bias=True)
Task 1 with 45 labels.
Dropout is Dropout(p=0.1, inplace=False)
Classifier layer is Linear(in_features=768, out_features=45, bias=True)
{'train_runtime': 2.8273, 'train_samples_per_second': 86.302, 'train_steps_per_second': 0.707, 'train_loss': 0.9978494644165039, 'epoch': 1.0}
Elapsed time for epoch 1: 0:00:02.839442
Evaluating on the validation dataset for task NER:
{'eval_loss': 0.39391154050827026, 'eval_runtime': 11.9184, 'eval_samples_per_second': 290.812, 'eval_steps_per_second': 2.349, 'epoch': 1.0}
Evaluating on the validation dataset for task POS:
{'eval_loss': 1.1675941944122314, 'eval_runtime': 20.1536, 'eval_samples_per_second': 249.98, 'eval_steps_per_second': 1.985, 'epoch': 1.0}
DEV MACRO LOSS FOR EPOCH 1: 0.7807528674602509


STARTING EPOCH 2
Resuming from checkpoint bert-base-cased-mtl/mtl_model_epoch1
Fou

In [99]:
model = TokenClassificationModel.from_pretrained('bert-base-cased-mtl/mtl_model_epoch2', local_files_only=True)
model.summarize_heads()

Found 0 heads


In [140]:
from sklearn.metrics import classification_report

def classification_report_on_test(trainer, task, name):
    print(f"Test classification report for task {name}:")
    num_labels = task.num_labels
    ds = Dataset.from_pandas(task.test_df)
    output = trainer.predict(ds)
    label_ids = output.label_ids.reshape(-1)
    predictions = output.predictions.reshape(-1, num_labels)
    predictions = np.argmax(predictions, axis=-1)
    mask = label_ids != ignore_index
    
    y_true = label_ids[mask]
    y_pred = predictions[mask]
    target_names = [task.index_to_label.get(ele, "") for ele in range(num_labels)]
    print(target_names)
    
    total = 0
    correct = 0
    for(t, p) in zip(y_true, y_pred):
        total = total + 1
        if t == p:
            correct = correct + 1
    accuracy = correct / total
    
    report = classification_report(
        y_true, y_pred,
        target_names=target_names
    )
    print(report)
    print(f'locally computed accuracy: {accuracy}')
    return accuracy


In [126]:
ner_acc = classification_report_on_test(trainer, ner_task, "NER")
pos_acc = classification_report_on_test(trainer, pos_task, "POS")
macro_acc = (ner_acc + pos_acc)/2
print(f"MTL macro accuracy: {macro_acc}")

Test classification report for task NER:
['B-PER', 'B-LOC', 'O', 'I-ORG', 'B-MISC', 'I-MISC', 'B-ORG', 'I-PER', 'I-LOC']
              precision    recall  f1-score   support

       B-PER       0.96      0.95      0.96      1617
       B-LOC       0.92      0.92      0.92      1668
           O       1.00      0.99      0.99     38554
       I-ORG       0.86      0.89      0.87       835
      B-MISC       0.76      0.82      0.79       702
      I-MISC       0.63      0.73      0.68       216
       B-ORG       0.89      0.88      0.89      1661
       I-PER       0.98      0.99      0.99      1156
       I-LOC       0.80      0.87      0.83       257

    accuracy                           0.98     46666
   macro avg       0.87      0.89      0.88     46666
weighted avg       0.98      0.98      0.98     46666

locally computed accuracy: 0.9784211203017186
Test classification report for task POS:
['FW', 'SYM', 'NNP', 'NN', 'VBN', 'JJ', ':', "''", '#', ',', 'VBG', 'IN', 'RP', 'PDT', 

/Users/msurdeanu/miniconda3/envs/transformers-mps/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/msurdeanu/miniconda3/envs/transformers-mps/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/msurdeanu/miniconda3/envs/transformers-mps/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa